In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import numpy as np
import time
from sqlalchemy import create_engine
import pyodbc

In [ ]:
headers = {'Authorization': 'Token token=""'}
indicator_id = '460'
start_date = '2017-01-01T00:00:00Z'
end_date = '2018-12-31T24:00:00Z'
#0a3ec68a89579911bd7610e20df0f79c83f79b10275b19025aaf414136c91f9a

In [ ]:
def get(indicator: str, start_date: str, end_date: str):

    root = f'https://api.esios.ree.es/indicators/{indicator}?&start_date={start_date}&end_date={end_date}/'

    data = requests.get(root,
                        headers=headers
                       )
    print(data.status_code)
    json = data.json()
    
    return json

In [ ]:
def df(json):
    
    table = pd.DataFrame()
    
    print("Converting JSON to DataFrame...")
    for val in tqdm(json['indicator']['values']):
        single_df = pd.DataFrame({'datetime': val['datetime_utc'], 'value': val['value']}, index=[0])
        table = table.append(single_df, ignore_index=True)

    table.insert(0, 'name', json['indicator']['name'])
    table.insert(1, 'id', json['indicator']['id'])
    table.insert(4, 'update_timestamp', json['indicator']['values_updated_at'])
    
    table.rename(columns={'value': 'demand_forecast'}, inplace=True)
    print("Conversion completed")
    
    return table

In [ ]:
prueba = get(indicator_id, '2017-01-01T00:00:00Z', '2018-12-31T24:00:00Z')

In [ ]:
df = df(prueba)

In [ ]:
df

In [ ]:
df['datetime'].min()

In [ ]:
speed_df = pd.read_csv("../DataOps_test/dataset_speed_optimization.csv")

In [ ]:
speed_df

In [ ]:
# Function t o a p p l y t o a l l rows i n t h e DataFrame

def function_to_apply(lat, lon):
    a = np.sin(lat/2)**2 + np.cos(lon) * np.sin(lon/2)**2
    
    return a

In [ ]:
df = speed_df.copy()

start = time.time()

list_results = []
for i in range(0, len(df)):
    r = function_to_apply(df.iloc[i]['latitude'], df.iloc[i]['longitude'])
    list_results.append(r)
    
df['distance'] = list_results

elapsed_time_fl = (time.time() - start)
print(elapsed_time_fl)

In [ ]:
df

In [ ]:
df = speed_df.copy()

start = time.time()

df['distance'] = np.sin(df['latitude']/2)**2 + np.cos(df['longitude']) * np.sin(df['longitude']/2)**2
elapsed_time_fl = (time.time() - start)

elapsed_time_fl

In [ ]:
for i in tqdm(range(0, len(df))):
    

In [ ]:
df

In [2]:
df = pd.read_csv("../data/david_blanco_etl.csv")

In [3]:
df.rename(columns={"datetime": "demand_datetime", "id":"indicator_id"}, inplace=True)
df.insert(0, 'id', [x+1 for x in range(len(df))])
for col in df.select_dtypes(include="object").columns.to_list():
    df[f'{col}'] = df[f'{col}'].astype('category')
df['name'] = "Prevision diaria de la demanda electrica peninsular"
df.columns

Index(['id', 'name', 'indicator_id', 'demand_datetime', 'demand_forecast',
       'update_timestamp'],
      dtype='object')

In [4]:
df.dtypes

id                     int64
name                  object
indicator_id           int64
demand_datetime     category
demand_forecast      float64
update_timestamp    category
dtype: object

In [5]:
server = "localhost"
database = "mock_db"
user = "sa"
password = "Snowdav3"

In [6]:
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server}; SERVER=localhost; Database=mock_db; UID=sa; PWD=Snowdav3;')
cursor = conn.cursor()

In [13]:
# Create Table

create_table = """
    CREATE TABLE demand_forecast_esios
    (
    id int PRIMARY KEY,
    name varchar(100),
    indicator_id int,
    demand_datetime varchar(100),
    demand_forecast int,
    update_timestamp varchar(100)
    )

"""
cursor.execute(create_table)
conn.commit()

In [9]:
cols = ','.join(df.columns.to_list())
cols

'id,name,indicator_id,demand_datetime,demand_forecast,update_timestamp'

In [10]:
sql = "INSERT INTO demand_forecast_esios ([id],[name],[indicator_id], [demand_datetime], [demand_forecast], [update_timestamp]) VALUES (?,?,?,?,?,?)"
sql_cols = df.columns.to_list()
cursor.executemany(sql, df[sql_cols].values.tolist())   
conn.commit()

17521it [00:08, 2111.48it/s]


In [ ]:
df.dtypes

In [ ]:
cursor.execute("DELETE FROM demand_forecast_esios")
conn.commit()

In [ ]:
df.select_dtypes(include="object").columns.to_list()

In [ ]:
df.dtypes

In [11]:
df

,id,name,indicator_id,demand_datetime,demand_forecast,update_timestamp
0,1,Prevision diaria de la demanda electrica penin...,460,2017-01-01T00:00:00Z,23616.0,2019-01-01T00:37:45.000+01:00
1,2,Prevision diaria de la demanda electrica penin...,460,2017-01-01T01:00:00Z,21893.0,2019-01-01T00:37:45.000+01:00
2,3,Prevision diaria de la demanda electrica penin...,460,2017-01-01T02:00:00Z,20693.0,2019-01-01T00:37:45.000+01:00
3,4,Prevision diaria de la demanda electrica penin...,460,2017-01-01T03:00:00Z,19599.0,2019-01-01T00:37:45.000+01:00
4,5,Prevision diaria de la demanda electrica penin...,460,2017-01-01T04:00:00Z,19211.0,2019-01-01T00:37:45.000+01:00
...,...,...,...,...,...,...
17516,17517,Prevision diaria de la demanda electrica penin...,460,2018-12-31T20:00:00Z,28025.0,2019-01-01T00:37:45.000+01:00
17517,17518,Prevision diaria de la demanda electrica penin...,460,2018-12-31T21:00:00Z,25633.0,2019-01-01T00:37:45.000+01:00
17518,17519,Prevision diaria de la demanda electrica penin...,460,2018-12-31T22:00:00Z,24442.0,2019-01-01T00:37:45.000+01:00
17519,17520,Prevision diaria de la demanda electrica penin...,460,2018-12-31T23:00:00Z,23755.0,2019-01-01T00:37:45.000+01:00


In [12]:
df.columns.to_list()

['id',
 'name',
 'indicator_id',
 'demand_datetime',
 'demand_forecast',
 'update_timestamp']